### 回归问题的评价指标（线性回归和多项式回归）

#### MSE 均方差

In [1]:
# 均方差MSE = 1/m * ∑(y_test_ - y_test)^2  与测试样本数M无关。

#### RMSE根方差

In [ ]:
# sqrt(MSE) 均方根误差。求根号的意义是保证mes的量纲和y一致。

#### 平均绝对误差 MAE

In [ ]:
# MAE 平均绝对误差 1/m * ∑|y_test_ - y_test|

#### R方值

In [2]:
# R^2 = 1 - SS_assum/SS_total = 1- ∑(y_ - y)^2/ ∑(y_mean - y)
# R^2 =1 分子为0，我们的预测模型没有错误，R方越大越好
# R^2 =0，相当于没有训练。
# R方小于0，我们的模型得到的结果还不如不训练的均值模型。可能说明数据之间没有线性关系

### 分类问题的评价标准（多分类和二分类）

#### 分类准确度

In [3]:
# print(sum(y_predict==y_test)/len(y_test))
# 缺点是，对于数据

In [4]:
# 对于极度偏斜（Skewed Data）的数据，只使用分类准确度是远远不够的；
# 不同类型的样本的数量的差距特别大；如该种癌症患者和健康人，比例为 1:1000 或者 1:10000；
# 面对这种极度偏斜的数据，分类准确度非常的高，其实算法是不够好的，甚至有些情况下非常烂的算法也能得到非常高的准确度；

#### 混淆矩阵

<img src= "./图片/混淆矩阵.png">

In [5]:
# TN、FP、FN、TP：表示预测结果的样本数量；
# TN（True Negative）：实际值为 Negative，预测值为 Negative，预测 negative 正确；
# FP（False Positive）：实际值为 Negative，预测值为 Positive，预测 Positive 错误；
# FN（False Negative）：实际值为 Positive，预测值为 Negative，预测 Negative 错误；
# TP（True Positive）：实际值为 Positive，预测值为 Positive，预测 Positive 正确；

#### 精准率和召回率

<img src = "./图片/精准率和召回率.png">

In [6]:
# 精准率和召回率：衡量分类算法坏话的指标，就是通过混淆矩阵所得；
# 对于有偏的数据的分类中，通常将 1 作为关注的事件，精准率就是指预测所关注的事件的准确率；
# 所关注的事件：人群中的癌症患者；

In [7]:
""" 精准率 """
# 精准率：预测所关注的事件的结果中（共预测了 20 次），预测正确的概率（8 次正确，12 次错误）；
# 精准率 = TP / (TP + FP) = 8 / (8 + 12) = 40%
# 含义：每做 100 次患病的预测，平均会有 40 次是正确的；

' 精准率 '

In [8]:
""" 召回率 """
# 召回率 = TP / (TP + FN) = 8 / (8 + 2) = 80%
# 解释：每当有 100 个癌症患者，通过该预测系统，能够成功的找出 80 个癌症患者；

' 召回率 '

In [9]:
# 精准率是针对预测结果的准确率而言
# 召回率是针对识别出关注事件的准确率

# 准确率并不能反映系统是否找出了全部的癌症病人，或者找出了多大比例的癌症病人；
# 而精准率和召回率更能直接的反应系统能够找出癌症病人的能力；

<img src = "./图片/精准率和召回率.jpg">

#### 实现混淆矩阵、精准率和召回率

In [10]:
import numpy as np
from sklearn import datasets

In [15]:
digits = datasets.load_digits()
X = digits.data
y = digits.target.copy()

# 创建极端数据，只关注数字9
y[digits.target == 9] = 1
y[digits.target != 9] = 0

(180,)

In [13]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X, y, random_state=666)

In [16]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)
log_reg.score(X_test,y_test)

0.9755555555555555

In [17]:
# 即使数据严重倾斜也有97%的准确率

In [44]:
# 计算TN、FP、FN、TP
def TN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    # 注意这里是 &
    return np.sum((y_true == 0) & (y_predict == 0))

def FP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    # 注意这里是 &
    return np.sum((y_true == 0) & (y_predict == 1))

def FN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    # 注意这里是 &
    return np.sum((y_true == 1) & (y_predict == 0))

def TP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    # 注意这里是 &
    return np.sum((y_true == 1) & (y_predict == 1))

In [45]:
TN(y_test,log_reg.predict(X_test))

403

In [46]:
TP(y_test,log_reg.predict(X_test))

36

In [47]:
# 混淆矩阵
def confusion_matrix(y_test, y_predict):
    return np.array([
        [TN(y_test, y_predict), FP(y_test, y_predict)],
        [FN(y_test, y_predict), TP(y_test, y_predict)]
    ])
confusion_matrix(y_test, log_reg.predict(X_test))

array([[403,   2],
       [  9,  36]])

In [48]:
# 精准率
def precision_score(y_true, y_predict):
    tp = TP(y_test, y_predict)
    fp = FP(y_test, y_predict)
    try:
        return tp / (tp + fp)
    except:
        return 0.0

In [49]:
precision_score(y_test, log_reg.predict(X_test))

0.9473684210526315

In [50]:
# 召回率
def recall_score(y_true, y_predict):
    tp = TP(y_true, y_predict)
    fn = FN(y_true, y_predict)
    try:
        return tp / (tp + fn)
    except:
        return 0.0

In [51]:
recall_score(y_test, log_reg.predict(X_test))

0.8

In [52]:
# 使用sklearn中的混淆矩阵、精准率、召回率

In [73]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


confusion_matrix(y_test, log_reg.predict(X_test))

array([[403,   2],
       [  9,  36]], dtype=int64)

In [74]:
precision_score(y_test, log_reg.predict(X_test))

0.9473684210526315

In [55]:
recall_score(y_test, log_reg.predict(X_test))

0.8

### 怎么评价这两个指标呢？

In [56]:
# 看具体应用，选择？


# 比如：股票预测而言，我们关注股价是否会升，升就意味着要投钱买股票。这种情况下，我们应该关注精准率
# 精准率越高越好，精准率表示，股价会升，而我们也预测对了，同时不必关系召回率，
# 召回率表示股价上升，被我们判错了，这种情况我们只是没赚钱，但是更重要的是没亏钱

# 对于，疾病预测系统而言，显然更关心召回率，也就是只要是得疾病的情况都应该识别出来。
# 而不必关心精确率，这是因为精确率是判断有病，如果真是这样，那我们再让这些病人做进一步的确诊就行。


# 所以召回率有点宁可错杀一千，不放过一人的意思。
# 而精确率则是，每次杀对的人。

In [57]:
# 如果同时关注精准率和召回率呢？  
# F1 score

#### F1 score

In [58]:
# F1 = 2 * Precision * recall / (precision + recall)   # 二者的调和平均值

In [59]:
# 调和平均值：如果 1/a = (1/b + 1/c) / 2，则称 a 是 b 和 c 的调和平均值；
# 调和平均值特点：|b - c| 越大，a 越小；当 b - c = 0 时，a = b = c，a 达到最大值；
# 具体到精准率和召回率，只有当二者大小均衡时，F1 指标才高

In [60]:
import numpy as np

In [61]:
def f1_score(precision_score, recall_score):
    try:
        return 2 * precision_score * recall_score / (precision_score + recall_score)
    except:
        return 0.0

In [62]:
precision_score = 0.5
recall_score = 0.5
f1_score(precision_score, recall_score)

0.5

In [75]:
precision_score = 0.1
recall_score = 0.9
f1_score(precision_score, recall_score)

0.18000000000000002

In [76]:
# 调和平均值的优势在于，只要有任何一个值很小，那么f1_score结果也很小
# 也就是只有两个指标都好的时候，f1_score才好

In [80]:
# 利用上面的样本数据
y_predict = log_reg.predict(X_test)

In [78]:
y_predict.shape

(450,)

In [82]:
from sklearn.metrics import f1_score

f1_score(y_test, y_predict)
# 0.8674698795180723
# 使用f1_score来衡量我们的算法准确率，才更准确。

0.8674698795180723

In [83]:
# 使用scikit-learn 中 sklearn.metrics 模块下的 confusion_matrix()、precision_score()、recall_score()、f1_score() 方法时，
# 所需要的参数都是 y_test、y_predict；

### Precision-Recall 平衡

In [84]:
# 通常精准率和召回率是相互牵制，互相矛盾的两个变量，不能同时增高

In [85]:
# 编程看看两这个变量的制约关系

In [87]:
# LogisticRegression() 类中的 predict() 方法中，默认阈值 threshold 为 0，再根据 decision_function() 
# 方法计算的待预测样本的 score 值进行对比分类：score < 0 分类结果为 0，score > 0 分类结果为 1；

# log_reg.decision_function(X_test)：计算所有待预测样本的 score 值，以向量的数量类型返回结果；
# 此处的 score 值不是概率值，是另一种判断分类的方式中样本的得分，根据样本的得分对样本进行分类；

log_reg.decision_function(X_test[:10])

array([-22.05700117, -33.02940957, -16.21334087, -80.3791447 ,
       -48.25125396, -24.54005629, -44.39168773, -25.04292757,
        -0.97829292, -19.7174399 ])

In [92]:
# 看下前10个样本值，预测结果都是0，因为他们的score值都小于0（decision_function得到的）
log_reg.predict(X[:10])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])